In [ ]:
# =============================
# STEP 1: Mount Google Drive
# =============================
# this script was done through colab and a file the zipped file of all the key frames were extracted

from google.colab import drive
import os
drive.mount('/content/drive')

# Path to your zip file on Drive
zip_path = "/content/drive/MyDrive/frames.zip"  # Update if different

# Path to unzip frames (also in Drive)
frames_dir = "/content/drive/MyDrive/rhythm_frames"  # Where frames will be extracted
os.makedirs(frames_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# =============================
# STEP 2: Unzip directly in Drive
# =============================
import zipfile
print(f"Unzipping {zip_path} to {frames_dir} ...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(frames_dir)

print("✅ Unzip complete.")
print("Sample videos in frames_dir:", os.listdir(frames_dir))


Unzipping /content/drive/MyDrive/frames.zip to /content/drive/MyDrive/rhythm_frames ...


KeyboardInterrupt: 

In [ ]:
# =============================
# STEP 3: Set output directory for keypoints
# =============================
output_dir = "/content/drive/MyDrive/rhythm_keypoints"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# =============================
# STEP 0: Setup GPU
# =============================
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

# =============================
# STEP 1: Install dependencies
# =============================
!pip install mediapipe opencv-python tqdm


Using device: cpu
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 30.8 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 60.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 32.7 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 91.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 33.1 MB/s  0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 35.3 MB/s  0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.

In [ ]:
# =============================
# STEP 3: Pose estimation on keyframes
# =============================
import mediapipe as mp
import cv2
import json
from tqdm import tqdm
import os

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
output_dir = "/content/pose_keypoints"
os.makedirs(output_dir, exist_ok=True)

# Adjust frames_dir to point to the actual directory containing video folders
# The original frames_dir was /content/drive/MyDrive/rhythm_frames
# And it appears that unzipping created a 'frames' sub-directory inside it.
frames_source_dir = os.path.join(frames_dir, 'frames')

# Iterate over videos
for vid_name in os.listdir(frames_source_dir):
    vid_path = os.path.join(frames_source_dir, vid_name)
    if not os.path.isdir(vid_path):
        continue

    vid_output_dir = os.path.join(output_dir, vid_name)
    os.makedirs(vid_output_dir, exist_ok=True)

    keypoints_json = {}

    for frame_file in tqdm(sorted(os.listdir(vid_path))):
        frame_path = os.path.join(vid_path, frame_file)
        image = cv2.imread(frame_path)

        # Add a check if the image was loaded successfully
        if image is None:
            print(f"Warning: Could not read image at {frame_path}. Skipping.")
            keypoints_json[frame_file] = None
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            keypoints = []
            for lm in results.pose_landmarks.landmark:
                keypoints.append({
                    "x": lm.x,
                    "y": lm.y,
                    "z": lm.z,
                    "visibility": lm.visibility
                })
            keypoints_json[frame_file] = keypoints
        else:
            keypoints_json[frame_file] = None

    # Save JSON per video
    with open(os.path.join(vid_output_dir, f"{vid_name}_keypoints.json"), "w") as f:
        json.dump(keypoints_json, f)

print("Pose estimation complete! Keypoints saved in:", output_dir)


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/720 [00:00<?, ?it/s]

100%|██████████| 720/720 [01:16<00:00,  9.40it/s]


100%|██████████| 720/720 [01:13<00:00,  9.82it/s]


0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 720/720 [01:16<00:00,  9.40it/s]


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 576/576 [01:05<00:00,  8.76it/s]


...

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 720/720 [01:16<00:00,  9.38it/s]


0it [00:00, ?it/s]
100%|██████████| 720/720 [01:07<00:00, 10.71it/s]


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 2094/2094 [03:32<00:00,  9.86it/s]


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1771/1771 [03:00<00:00,  9.80it/s]


Pose estimation complete! Keypoints saved in: /content/pose_keypoints


In [ ]:
import cv2
img = cv2.imread("/content/drive/MyDrive/rhythm_frames/frames/gWA_sFM_cAll_d26_mWA5_ch13/frame_000000.jpg")
print(img.shape)  # Check if image was read


(1080, 1920, 3)


In [ ]:
import os

frame_path = "/content/drive/MyDrive/rhythm_frames/frames/gWA_sFM_cAll_d26_mWA5_ch13/frame_000000.jpg"
print("Exists?", os.path.exists(frame_path))
print("Is file?", os.path.isfile(frame_path))


Exists? True
Is file? True


In [ ]:
import mediapipe as mp
import cv2
import json

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Path to your single frame (testing on one frame)
frame_path = "/content/drive/MyDrive/rhythm_frames/frames/gWA_sFM_cAll_d26_mWA5_ch13/frame_000000.jpg"

# Verify file
import os
if not os.path.isfile(frame_path):
    raise FileNotFoundError(f"File not found: {frame_path}")

# Read image
image = cv2.imread(frame_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Run pose estimation
results = pose.process(image_rgb)

# Extract keypoints
if results.pose_landmarks:
    keypoints = []
    for lm in results.pose_landmarks.landmark:
        keypoints.append({
            "x": lm.x,
            "y": lm.y,
            "z": lm.z,
            "visibility": lm.visibility
        })
    print("Pose keypoints detected:")
    print(keypoints)
else:
    print("No pose detected in this image.")


Pose keypoints detected:
[{'x': 0.4911632537841797, 'y': 0.4503205418586731, 'z': -0.03365657851099968, 'visibility': 0.9997977614402771}, {'x': 0.4937855899333954, 'y': 0.4433531165122986, 'z': -0.026592375710606575, 'visibility': 0.9996738433837891}, {'x': 0.49529579281806946, 'y': 0.44351065158843994, 'z': -0.026613621041178703, 'visibility': 0.9996862411499023}, {'x': 0.4967757761478424, 'y': 0.4436790943145752, 'z': -0.026628078892827034, 'visibility': 0.9997246861457825}, {'x': 0.4884672164916992, 'y': 0.4438886046409607, 'z': -0.024615637958049774, 'visibility': 0.9995887875556946}, {'x': 0.4865288734436035, 'y': 0.4443700909614563, 'z': -0.024625595659017563, 'visibility': 0.9996117949485779}, {'x': 0.4848006069660187, 'y': 0.4449719786643982, 'z': -0.02460903488099575, 'visibility': 0.9996446371078491}, {'x': 0.5006296038627625, 'y': 0.44807353615760803, 'z': 0.014562777243554592, 'visibility': 0.999700665473938}, {'x': 0.48310595750808716, 'y': 0.4510539770126343, 'z': 0.0240

In [ ]:
import mediapipe as mp
import cv2
import json
import os
from tqdm import tqdm

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Folder containing frames for one video
video_frames_dir = "/content/drive/MyDrive/rhythm_frames/frames/gWA_sFM_cAll_d26_mWA5_ch13"

# Output folder for keypoints
output_dir = "/content/pose_keypoints_single_video"
os.makedirs(output_dir, exist_ok=True)

keypoints_json = {}

# Iterate over frames
for frame_file in tqdm(sorted(os.listdir(video_frames_dir))):
    frame_path = os.path.join(video_frames_dir, frame_file)
    if not os.path.isfile(frame_path):
        continue

    # Read and process image
    image = cv2.imread(frame_path)
    if image is None:
        print(f"Warning: Could not read image {frame_path}. Skipping.")
        keypoints_json[frame_file] = None
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    # Extract keypoints
    if results.pose_landmarks:
        keypoints = []
        for lm in results.pose_landmarks.landmark:
            keypoints.append({
                "x": lm.x,
                "y": lm.y,
                "z": lm.z,
                "visibility": lm.visibility
            })
        keypoints_json[frame_file] = keypoints
    else:
        keypoints_json[frame_file] = None

# Save JSON for this video
output_file = os.path.join(output_dir, "gWA_sFM_cAll_d26_mWA5_ch13_keypoints.json")
with open(output_file, "w") as f:
    json.dump(keypoints_json, f, indent=2)

print("Pose estimation complete! Keypoints saved at:", output_file)


100%|██████████| 1771/1771 [03:06<00:00,  9.51it/s]


Pose estimation complete! Keypoints saved at: /content/pose_keypoints_single_video/gWA_sFM_cAll_d26_mWA5_ch13_keypoints.json


In [ ]:
import os

def get_remaining_folders(path):
    remaining = []
    for root, dirs, files in os.walk(path):
        for d in dirs:
            dir_path = os.path.join(root, d)
            # Check if the folder is not empty
            if os.listdir(dir_path):
                remaining.append(dir_path)
    return remaining

frames_dir = "/content/drive/MyDrive/rhythm_frames/frames"
remaining_folders = get_remaining_folders(frames_dir)

# Print all remaining folders
for folder in remaining_folders:
    print(folder)

# Print total count
print(f"\nTotal remaining folders: {len(remaining_folders)}")


/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d04_mBR0_ch06
/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d04_mBR0_ch07
/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d04_mBR0_ch10
/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d04_mBR2_ch01
/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d04_mBR3_ch05
/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d04_mBR3_ch08
/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d05_mBR4_ch01
/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d05_mBR4_ch10
/content/drive/MyDrive/rhythm_frames/frames/gBR_sBM_cAll_d05_mBR5_ch05
/content/drive/MyDrive/rhythm_frames/frames/gBR_sFM_cAll_d04_mBR2_ch03
/content/drive/MyDrive/rhythm_frames/frames/gBR_sFM_cAll_d04_mBR3_ch04
/content/drive/MyDrive/rhythm_frames/frames/gHO_sBM_cAll_d19_mHO0_ch01
/content/drive/MyDrive/rhythm_frames/frames/gHO_sBM_cAll_d19_mHO1_ch02
/content/drive/MyDrive/rhythm_frames/frames/gHO_sBM_cAll_d19_mHO1_ch03
/conte

In [ ]:
import os
import cv2
import mediapipe as mp
import json
from tqdm import tqdm

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Directories
frames_dir = "/content/drive/MyDrive/rhythm_frames/frames"
output_dir = "/content/pose_keypoints_attempt2"
os.makedirs(output_dir, exist_ok=True)

# Get all non-empty folders
all_folders = [os.path.join(frames_dir, d) for d in os.listdir(frames_dir)
               if os.path.isdir(os.path.join(frames_dir, d)) and os.listdir(os.path.join(frames_dir, d))]

# Limit to first 20
folders_to_process = all_folders[:20]

for vid_path in folders_to_process:
    vid_name = os.path.basename(vid_path)
    vid_output_dir = os.path.join(output_dir, vid_name)
    os.makedirs(vid_output_dir, exist_ok=True)

    keypoints_json = {}

    for frame_file in tqdm(sorted(os.listdir(vid_path))):
        frame_path = os.path.join(vid_path, frame_file)
        image = cv2.imread(frame_path)
        if image is None:
            print(f"Warning: Could not read image at {frame_path}. Skipping.")
            keypoints_json[frame_file] = None
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            keypoints = []
            for lm in results.pose_landmarks.landmark:
                keypoints.append({
                    "x": lm.x,
                    "y": lm.y,
                    "z": lm.z,
                    "visibility": lm.visibility
                })
            keypoints_json[frame_file] = keypoints
        else:
            keypoints_json[frame_file] = None

    # Save JSON per video
    with open(os.path.join(vid_output_dir, f"{vid_name}_keypoints.json"), "w") as f:
        json.dump(keypoints_json, f)

print("Pose estimation complete on first 20 folders!")


  0%|          | 2/720 [00:00<00:59, 11.99it/s]

100%|██████████| 720/720 [00:59<00:00, 12.03it/s]


100%|██████████| 720/720 [01:09<00:00, 10.33it/s]


100%|██████████| 720/720 [01:14<00:00,  9.72it/s]


100%|██████████| 576/576 [00:57<00:00,  9.99it/s]


...

100%|██████████| 481/481 [00:43<00:00, 10.94it/s]


100%|██████████| 481/481 [00:43<00:00, 10.97it/s]


100%|██████████| 461/461 [00:42<00:00, 10.90it/s]


100%|██████████| 427/427 [00:41<00:00, 10.35it/s]


Pose estimation complete on first 20 folders!


In [ ]:
import os
import cv2
import mediapipe as mp
from tqdm import tqdm

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Directories
frames_dir = "/content/drive/MyDrive/rhythm_frames/frames"
output_dir = "/content/pose_overlay_frames"
os.makedirs(output_dir, exist_ok=True)

# Get all non-empty folders and take only the first one
all_folders = [os.path.join(frames_dir, d) for d in os.listdir(frames_dir)
               if os.path.isdir(os.path.join(frames_dir, d)) and os.listdir(os.path.join(frames_dir, d))]

if len(all_folders) == 0:
    raise ValueError("No non-empty folders found in frames_dir")

vid_path = all_folders[0]  # first folder
vid_name = os.path.basename(vid_path)
vid_output_dir = os.path.join(output_dir, vid_name)
os.makedirs(vid_output_dir, exist_ok=True)

for frame_file in tqdm(sorted(os.listdir(vid_path))):
    frame_path = os.path.join(vid_path, frame_file)
    image = cv2.imread(frame_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Draw skeleton on the image
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=3),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
        )

    # Save overlaid frame
    cv2.imwrite(os.path.join(vid_output_dir, frame_file), image)

print(f"Overlay complete for {vid_name}!")


100%|██████████| 720/720 [01:16<00:00,  9.45it/s]

Overlay complete for gBR_sBM_cAll_d04_mBR0_ch06!


In [ ]:
import os
import cv2
import mediapipe as mp
import random
from tqdm import tqdm

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Directories
frames_dir = "/content/drive/MyDrive/rhythm_frames/frames"
output_dir = "/content/pose_overlay_random_frames"
os.makedirs(output_dir, exist_ok=True)

# Get all non-empty folders
all_folders = [os.path.join(frames_dir, d) for d in os.listdir(frames_dir)
               if os.path.isdir(os.path.join(frames_dir, d)) and os.listdir(os.path.join(frames_dir, d))]

# Take the next two folders (skip the first one)
folders_to_process = all_folders[1:3]

for vid_path in folders_to_process:
    vid_name = os.path.basename(vid_path)
    vid_output_dir = os.path.join(output_dir, vid_name)
    os.makedirs(vid_output_dir, exist_ok=True)

    # Choose one random frame
    frame_file = random.choice(sorted(os.listdir(vid_path)))
    frame_path = os.path.join(vid_path, frame_file)
    image = cv2.imread(frame_path)
    if image is None:
        print(f"Could not read {frame_file}, skipping.")
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Draw skeleton on the image
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=3),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
        )

    # Save overlaid frame
    cv2.imwrite(os.path.join(vid_output_dir, frame_file), image)
    print(f"Overlay done for {vid_name}, frame: {frame_file}")

print("Random frame overlay complete for next two videos!")


Overlay done for gBR_sBM_cAll_d04_mBR0_ch07, frame: frame_000098.jpg
Overlay done for gBR_sBM_cAll_d04_mBR0_ch10, frame: frame_000048.jpg
Random frame overlay complete for next two videos!
